In [27]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def calculate_entropy(self, y):
        #find probability of both classes i.e 0 and 1
        unique_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        entropy = -np.sum(probabilities * np.log2(probabilities))
        return entropy

    def calculate_information_gain(self, X, y, feature_idx):
        # Calculate entropy before the split
        total_entropy = self.calculate_entropy(y)

        # Calculate entropy after the split
        unique_values = np.unique(X[:, feature_idx])
        weighted_entropy = 0

        for value in unique_values:
            subset_y = y[X[:, feature_idx] == value]
            weighted_entropy += (len(subset_y) / len(y)) * self.calculate_entropy(subset_y)

        information_gain = total_entropy - weighted_entropy
        return information_gain

    def find_best_split(self, X, y, features):
        best_feature = None
        best_information_gain = -1

        for feature_idx in features:
            information_gain = self.calculate_information_gain(X, y, feature_idx)

            if information_gain > best_information_gain:
                best_feature = feature_idx
                best_information_gain = information_gain

        return best_feature

    def fit(self, X, y, depth=0, features=None):
        if features is None:
            features = list(range(X.shape[1]))

        # Check termination conditions
        if len(np.unique(y)) == 1:
            # Pure node, all samples have the same class
            return {'class': y[0]}

        if depth == self.max_depth:
            # Maximum depth reached
            return {'class': np.argmax(np.bincount(y))}

        if not features:
            # No more features to split on
            return {'class': np.argmax(np.bincount(y))}

        # Find the best feature to split on
        best_feature = self.find_best_split(X, y, features)

        if best_feature is None:
            # Unable to find a split
            return {'class': np.argmax(np.bincount(y))}

        # Split the data based on the best feature
        unique_values = np.unique(X[:, best_feature])
        node = {'feature': best_feature, 'children': {}}

        for value in unique_values:
            # Create a child node for each unique value of the best feature
            subset_X = X[X[:, best_feature] == value]
            subset_y = y[X[:, best_feature] == value]
            subset_features = [f for f in features if f != best_feature]

            node['children'][value] = self.fit(subset_X, subset_y, depth + 1, subset_features)

        return node

    def predict_instance(self, x, tree):
        if 'class' in tree:
            return tree['class']
        feature_value = x[tree['feature']]
        if feature_value not in tree['children']:
            return np.argmax(np.bincount(y))  # Predict the majority class
        return self.predict_instance(x, tree['children'][feature_value])

    def predict(self, X):
        return [self.predict_instance(x, self.tree) for x in X]

# Example usage:
# Assuming X_train and y_train are your training data and labels
X_train = np.array([[0, 1, 1, 0, 0],
                    [1, 0, 0, 1, 1],
                    [1, 1, 1, 0, 0],
                    [0, 0, 1, 1, 1],
                    [0, 1, 0, 0, 1],
                    [1, 0, 1, 1, 0],
                    [1, 1, 0, 0, 0],
                    [0, 0, 0, 1, 1],
                    [1, 0, 0, 1, 0],
                    [0, 1, 1, 0, 1]])
y_train = np.array([0, 1, 1, 0, 1, 0, 1, 0, 1, 0])

# Create and train the decision tree
tree = DecisionTree(max_depth=5)
tree.tree = tree.fit(X_train, y_train)
print("Printing Tree : ")
print(tree.tree)
print()
# Example prediction
X_test = np.array([[0, 1, 1, 0, 1],
                   [1, 0, 1, 1, 0]])
predictions = tree.predict(X_test)

print("Predictions:", predictions)


Printing Tree : 
{'feature': 0, 'children': {0: {'feature': 2, 'children': {0: {'feature': 1, 'children': {0: {'class': 0}, 1: {'class': 1}}}, 1: {'class': 0}}}, 1: {'feature': 2, 'children': {0: {'class': 1}, 1: {'feature': 1, 'children': {0: {'class': 0}, 1: {'class': 1}}}}}}}

Predictions: [0, 0]
